In [1]:
import pandas as pd
import numpy as np
import sklearn.feature_selection as fs

np.set_printoptions(precision=3)

# Feature Selection
* reduce overfitting (menos ruido)
* mejora precisión
* reduce tiempos de entrenamiento


Feature se traduce a caracteristicas, muchas veces les dire por el nombre en ingles ya que te sera mas facil comprender la literatura en linea.

Caracteristicas irrelevantes pueden entregarnos un mal rendimiento de nuestro modelo, a veces más data puede resultar contraproducente; por ello no siempre podemos ingresar todo a la caja negra y esperar a que ocurra algo magico. Piensalo como que estas diluyendo los datos importantes y el modelo tiene que trabajar extra o tener suerte para obtener la información realmente importante.

Utilizaremos herramientas de scikit-learn

https://scikit-learn.org/stable/modules/feature_selection.html


# Seleccion Univariada
Usar un test estadistico para elegir aquellas con una relacion al output, scikit utiliza `SelectKBest`; aqui utlizaremos uno de los text que contiene, el cual es: `F-test` lo cual computa la tabla ANOVA-F 

In [ ]:
# download the dataset from https://github.com/jbrownlee/Datasets/blob/master/pima-indians-diabetes.data.csv and save it in the same folder as this notebook

csv_url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv'
pd.read_csv(csv_url, header=None).to_csv('pima-indians-diabetes.data.csv', index=False, header=False)


In [5]:
filename = 'pima-indians-diabetes.data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = pd.read_csv(filename, names=names)
# extraemos en un arreglo los valores (algo como una matriz)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]

dataframe.head()

,preg,plas,pres,skin,test,mass,pedi,age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


### ANOVA
analisis de varianza, lo que utilizamos en el `f_classif` es observar la cantidad de clasificaciones entregadas por `Y` nuestro target vector, en este caso es que el guinea pig tenga diabetes o no, `1` o `0`, esto quiere decir que dividiremos los datos en 2 grupos;

##### F
Se encarga de poner la hipotesis, 
- la nula es que las medias de los grupos son iguales.
- la alternativa es que almenos una media entre los grupos es distinta

En este caso la funcion se encarga de separar las caracteristicas, por lo que solo nos encargamos por ver el puntaje F de cada caracteristica, 
- aqui para cada feature:
    * se calcula la variabilidad
    * se compara la variabilidad entre grupos
    * se obtiene asi el F-test (es un test de hipotesis al final)

el puntaje F es:
$$
F = \frac{\text{variabilidad entre grupos}}{\text{variabilidad dentro del grupo}}
$$

un mayor puntaje $F$ quiere decir que existe una mayor evidencia de que la `feature` o `caracteristica` tiene influencia en la diferencia de los grupos; en este caso un alto puntaje $F$ es que la caracteristica tiene evidencia de estar relacionada entre que el guineapig tenga diabetes o no

##### p
Luego el puntaje p es la probabilidad de haber obtenido este resultado por suerte, un puntaje p cercano a 0, se suele tomar como puntaje p suficiente uno menor a 0.05

En el notebook de `2_Estadistica/hipotesis_testing/F-test` puedes encontrar mayor informacion sobre el funcionamiento interno, lo recomiendo pues no debemos de aplicar estadistica sin entender que es lo que estamos haciendo:

*es mejor una respuesta correcta aproximada, que una respuesta incorrecta exacta*

la estadistica nos entrega respuestas, es facil obtener respuestas, lo dificil es entender que son esas respuestas.

In [12]:
# ejemplo del F test
# esto es un analisis de varianza, computa el F-statistic de cada feature
# y el p-score de cada feature
Fstatistic, Pscore = fs.f_classif(X,Y)

# print as a table
print('F-statistic\tP-score\t\tFeature')   
for i in range(len(names)-1):
    print('{:.3f}\t\t{:.3f}\t\t{}'.format(Fstatistic[i], Pscore[i], names[i]))

F-statistic	P-score		Feature
39.670		0.000		preg
213.162		0.000		plas
3.257		0.072		pres
4.304		0.038		skin
13.281		0.000		test
71.772		0.000		mass
23.871		0.000		pedi
46.141		0.000		age


In [15]:
# utilizamos el test F para seleccionar las 4 mejores caracteristicas
test = fs.SelectKBest(score_func= fs.f_classif, k=4)
fit = test.fit(X, Y)

# summarize scores
print(fit.scores_)
features = fit.transform(X)
# summarize selected features
print(features[0:5,:])

[ 39.67  213.162   3.257   4.304  13.281  71.772  23.871  46.141]
[[  6.  148.   33.6  50. ]
 [  1.   85.   26.6  31. ]
 [  8.  183.   23.3  32. ]
 [  1.   89.   28.1  21. ]
 [  0.  137.   43.1  33. ]]


In [48]:
# get the index of the 4 maximum scores
features_index = fit.scores_.argsort()[-4:][::-1]
print('Indices importantes')
print(features_index)

# get the index [1, 5, 7, 0] of names
names_index = [names[idx] for idx in features_index]
print(names_index)
print(X[:5, features_index])


Indices importantes
[1 5 7 0]
['plas', 'mass', 'age', 'preg']
[[148.   33.6  50.    6. ]
 [ 85.   26.6  31.    1. ]
 [183.   23.3  32.    8. ]
 [ 89.   28.1  21.    1. ]
 [137.   43.1  33.    0. ]]


In [53]:
# asi reconstruyendo un data frame ahora con informacion de las 4 mejores caracteristicas
df = pd.DataFrame(features, columns=names_index)
df['class'] = Y
df.head()

,plas,mass,age,preg,class
0,6.0,148.0,33.6,50.0,1.0
1,1.0,85.0,26.6,31.0,0.0
2,8.0,183.0,23.3,32.0,1.0
3,1.0,89.0,28.1,21.0,0.0
4,0.0,137.0,43.1,33.0,1.0


In [ ]:
df.to_csv('pima-indians-diabetes-selected.csv')

# Conclusion
Hemos concentrado asi la informacion, dejando aquellos datos que tiene mayor influencia en la clase, ahora el tiempo de entreamiento asi como la precision sera mucho menor; intenta utilizar ahora un modelo de regresion con lo que haz visto en el notebook `1_main.ipynb`, lee el codigo, copia, pega y rearma codigo que sea capaz de predecir si estos probes conejillos de india tienen diabetes:

He aqui un simple ejemplo si te sientes perdidx:

In [73]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

In [ ]:
# en caso de que guardaras el archivo y comeinces de nuevo
# df = pd.read_csv('pima-indians-diabetes-selected.csv')

In [74]:
array = df.values
print(array[:,0:-1][:5])
print(array[:,-1][:5])

X = array[:,0:-1]
Y = array[:,-1]


[[  6.  148.   33.6  50. ]
 [  1.   85.   26.6  31. ]
 [  8.  183.   23.3  32. ]
 [  1.   89.   28.1  21. ]
 [  0.  137.   43.1  33. ]]
[1. 0. 1. 0. 1.]


In [75]:
validation_size = 0.20

X_train, X_validation, Y_train, Y_validation = train_test_split(
                X, Y, test_size=validation_size, random_state=1)


In [76]:
model = SVC(gamma='auto')
model.fit(X_train, Y_train)
predictions = model.predict(X_validation)

In [79]:
print(f"Precision utilizando SupportVectorMachine: {accuracy_score(Y_validation, predictions):.3f}")

# crees poder mejorar la precision utilizando otro modelo?

Precision utilizando SupportVectorMachine: 0.636
